# A short intro on deploying fastai models to Google Cloud functions

https://github.com/ttumiel/gcloud-functions-fastai

In [0]:
%load_ext autoreload
%autoreload 2

## Train a Movie Classifier

In [0]:
from fastai.text import *

In [0]:
bs = 64
path = untar_data(URLs.IMDB_SAMPLE)

data_lm = (TextList.from_csv(path, "texts.csv", cols=["text"])
            .split_by_rand_pct(0.1)
            .label_for_lm()           
            .databunch(bs=bs))
data_lm.save('data_lm.pkl')

In [4]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))
learn.save_encoder('fine_tuned_enc')

epoch,train_loss,valid_loss,accuracy,time
0,4.200028,3.914214,0.286129,00:11


In [0]:
data_clas = (TextList.from_csv(path, "texts.csv", vocab=data_lm.vocab, cols=["text"])
             .split_from_df(2)
             .label_from_df(0)
             .databunch(bs=bs))

In [6]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.617859,0.646960,0.695000,00:07


In [7]:
learn.predict("I really hated that movie, it was awful!")

(Category negative, tensor(0), tensor([0.5130, 0.4870]))

## Test function that will be deployed

In [9]:
!git clone https://github.com/ttumiel/gcloud-functions-fastai gcf

Cloning into 'gcf'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 60 (delta 27), reused 58 (delta 25), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [0]:
learn.export("/content/gcf/data/export.pkl")

In [13]:
%cd gcf

/content/gcf


In [14]:
from test import test_function
from main import text_clas
json_data = {"data": "the movie was very good"}
test_function(text_clas, json_data)

{'category': 'positive',
 'position': 1,
 'preds': [0.45273223519325256, 0.5472677946090698]}

## Deploy to gcloud functions

To deploy your function, you will need a Google Cloud account with billing enabled. When you run `gcloud init`, you will have to log in to your account and create a new "project" for the deployment.

1. Choose `[1] Re-initialize this configuration [default] with new settings`
2. Type `Y`
3. After logging int, copy and paste the verification code in the box.
4. Create a new project noting down the region and the project name.

In [0]:
!gcloud init

In [0]:
# Deployment can take several minutes
!gcloud functions deploy text_clas --runtime python37 --trigger-http --memory 512MB

Copy the URL from the output above and make a request. You can provide input data via two methods by default:

- As a query string with the key `data`
- As part of json data in a post request

In [21]:
!curl https://REGION-PROJECTNAME.cloudfunctions.net/text_clas?data=that%20movie%20was%20great

{"category":"positive","position":1,"preds":[0.40685057640075684,0.5931494235992432]}


In [29]:
!curl --data "data=the movie was great" https://REGION-PROJECTNAME.cloudfunctions.net/text_clas

{"category":"positive","position":1,"preds":[0.40983322262763977,0.5901667475700378]}
